In [83]:
import openai

In [84]:
from dotenv import load_dotenv

In [85]:
load_dotenv("/home/tom/two/envapi/my-env")

True

In [86]:
persistent_directory="/home/tom/two/vstore/chroma/"

In [87]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings

In [88]:
embeddings = OpenAIEmbeddings()

In [89]:
vectordb = Chroma(
    persist_directory=persistent_directory,
    embedding_function=embeddings
)

In [90]:
print(vectordb._collection.count())

209


In [91]:
texts = [
    """The Amanita phalloides has a large and imposing epigeous (aboveground) fruiting body (basidiocarp).""",
    """A mushroom with a large fruiting body is the Amanita phalloides. Some varieties are all-white.""",
    """A. phalloides, a.k.a Death Cap, is one of the most poisonous of all known mushrooms.""",
]

In [92]:
smalldb = Chroma.from_texts(texts, embedding=embeddings)

In [93]:
smalldb

In [98]:
question = "Tell me about all-white mushrooms with large fruiting bodies"

In [100]:
smalldb.similarity_search(question, k=2)

[Document(page_content='A mushroom with a large fruiting body is the Amanita phalloides. Some varieties are all-white.'),
 Document(page_content='A mushroom with a large fruiting body is the Amanita phalloides. Some varieties are all-white.')]

In [101]:
 smalldb.max_marginal_relevance_search(question, k=2, fetch_k=3)

[Document(page_content='A mushroom with a large fruiting body is the Amanita phalloides. Some varieties are all-white.'),
 Document(page_content='A mushroom with a large fruiting body is the Amanita phalloides. Some varieties are all-white.')]

In [102]:
question = "what did they say about matlab?"

In [103]:
doc_ss = vectordb.similarity_search(question, k=3)

In [104]:
doc_ss[0].page_content[:100]

'those homeworks will be done in either MATLA B or in Octave, which is sort of — I \nknow some people '

In [105]:
doc_ss[1].page_content[:100]

'those homeworks will be done in either MATLA B or in Octave, which is sort of — I \nknow some people '

In [106]:
docs_mmr = vectordb.max_marginal_relevance_search(question, k=3)

In [107]:
docs_mmr[0].page_content[:100]

'those homeworks will be done in either MATLA B or in Octave, which is sort of — I \nknow some people '

In [108]:
docs_mmr[1].page_content[:100]

'algorithm then? So what’s different? How come  I was making all that noise earlier about \nleast squa'

In [109]:
#now run the old question 
question = "what did say about regression in the thrid lecture?"

In [110]:
#done manually by specifying the metadata filter

In [111]:
docs = vectordb.similarity_search(
    question, 
    k=3,
    filter={'source': 'https://see.stanford.edu/materials/aimlcs229/transcripts/machinelearning-lecture03.pdf'}
)

In [112]:
for d in docs:
    print(d.metadata)

{'page': 0, 'source': 'https://see.stanford.edu/materials/aimlcs229/transcripts/machinelearning-lecture03.pdf'}
{'page': 14, 'source': 'https://see.stanford.edu/materials/aimlcs229/transcripts/machinelearning-lecture03.pdf'}
{'page': 6, 'source': 'https://see.stanford.edu/materials/aimlcs229/transcripts/machinelearning-lecture03.pdf'}


In [26]:
#now all the results are from 3rd lecture only, as expected

In [54]:
#now lets do the same using LLMs, 

In [113]:
from langchain.llms import OpenAI
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo

In [114]:
metadata_field_info = [
    AttributeInfo(
        name="source",
        description="The lecture the chunk is from, should be one of `docs/cs229_lectures/MachineLearning-Lecture01.pdf`, `docs/cs229_lectures/MachineLearning-Lecture02.pdf`, or `docs/cs229_lectures/MachineLearning-Lecture03.pdf`",
        type="string",
    ),
    AttributeInfo(
        name="page",
        description="The page from the lecture",
        type="integer",
    ),
]

In [115]:
document_content_description="Lecture Notes"

In [116]:
llm = OpenAI(temperature=0)

In [117]:
retriever = SelfQueryRetriever.from_llm(
    llm,
    vectordb,
    document_content_description,
    metadata_field_info,
    verbose=True
)

In [118]:
docs = retriever.get_relevant_documents(question)

In [119]:
len(docs)

0

In [121]:
#retriever

In [122]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

In [123]:
llm = OpenAI(temperature=0)

In [124]:
compressor = LLMChainExtractor.from_llm(llm)

In [125]:
compression_retirevor = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=vectordb.as_retriever()
)

In [126]:
question="what did they say about matlab?"

In [127]:
compressed_docs = compression_retirevor.get_relevant_documents(question)
compressed_docs

/media/tom/StorageTWO/python-venvs/botvenv/lib/python3.10/site-packages/langchain/chains/llm.py:321: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/media/tom/StorageTWO/python-venvs/botvenv/lib/python3.10/site-packages/langchain/chains/llm.py:321: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/media/tom/StorageTWO/python-venvs/botvenv/lib/python3.10/site-packages/langchain/chains/llm.py:321: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/media/tom/StorageTWO/python-venvs/botvenv/lib/python3.10/site-packages/langchain/chains/llm.py:321: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 

[Document(page_content='"MATLAB is I guess part of the programming language that makes it very easy to write codes using matrices, to write code for numerical routines, to move data around, to plot data. And it\'s sort of an extremely easy to learn tool to use for implementing a lot of learning algorithms."', metadata={'page': 8, 'source': 'https://see.stanford.edu/materials/aimlcs229/transcripts/machinelearning-lecture01.pdf'}),
 Document(page_content='"MATLAB is I guess part of the programming language that makes it very easy to write codes using matrices, to write code for numerical routines, to move data around, to plot data. And it\'s sort of an extremely easy to learn tool to use for implementing a lot of learning algorithms."', metadata={'page': 8, 'source': 'https://see.stanford.edu/materials/aimlcs229/transcripts/machinelearning-lecture01.pdf'}),
 Document(page_content='"And the student said, "Oh, it was the MATLAB." So for those of you that don\'t know MATLAB yet, I hope you 

In [131]:
for i,d in enumerate(compressed_docs):
    print(f'Document {i} ----------------->')
    print(d.page_content)

Document 0 ----------------->
"MATLAB is I guess part of the programming language that makes it very easy to write codes using matrices, to write code for numerical routines, to move data around, to plot data. And it's sort of an extremely easy to learn tool to use for implementing a lot of learning algorithms."
Document 1 ----------------->
"MATLAB is I guess part of the programming language that makes it very easy to write codes using matrices, to write code for numerical routines, to move data around, to plot data. And it's sort of an extremely easy to learn tool to use for implementing a lot of learning algorithms."
Document 2 ----------------->
"And the student said, "Oh, it was the MATLAB." So for those of you that don't know MATLAB yet, I hope you do learn it. It's not hard, and we'll actually have a short MATLAB tutorial in one of the discussion sections for those of you that don't know it."
Document 3 ----------------->
"And the student said, "Oh, it was the MATLAB." So for th

In [132]:
compression_retirevor = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=vectordb.as_retriever(search_type='mmr')
)

In [133]:
compressed_docs = compression_retirevor.get_relevant_documents(question)


/media/tom/StorageTWO/python-venvs/botvenv/lib/python3.10/site-packages/langchain/chains/llm.py:321: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/media/tom/StorageTWO/python-venvs/botvenv/lib/python3.10/site-packages/langchain/chains/llm.py:321: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/media/tom/StorageTWO/python-venvs/botvenv/lib/python3.10/site-packages/langchain/chains/llm.py:321: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/media/tom/StorageTWO/python-venvs/botvenv/lib/python3.10/site-packages/langchain/chains/llm.py:321: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 

In [134]:
for i,d in enumerate(compressed_docs):
    print(f'Document {i} ----------------->')
    print(d.page_content)

Document 0 ----------------->
"MATLAB is I guess part of the programming language that makes it very easy to write codes using matrices, to write code for numerical routines, to move data around, to plot data. And it's sort of an extremely easy to learn tool to use for implementing a lot of learning algorithms."
Document 1 ----------------->
"And the student said, "Oh, it was the MATLAB." So for those of you that don't know MATLAB yet, I hope you do learn it. It's not hard, and we'll actually have a short MATLAB tutorial in one of the discussion sections for those of you that don't know it."


In [135]:
#Other types of retrieval i.e. SVMRetriever, TfIDFRetriever etc

In [136]:
from langchain.retrievers import SVMRetriever
from langchain.retrievers import TFIDFRetriever
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [137]:
loader = PyPDFLoader("https://see.stanford.edu/materials/aimlcs229/transcripts/machinelearning-lecture01.pdf")
pages = loader.load()

In [138]:
all_page_text = [p.page_content for p in pages]

In [140]:
len(all_page_text)

22

In [141]:
joined_page_text = " ".join(all_page_text)

In [144]:
#joined_page_text
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=150)
splits = text_splitter.split_text(joined_page_text)

In [145]:
svm_retriever = SVMRetriever.from_texts(splits, embeddings)

Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-embedding-ada-002 in organization org-pVOnO8dpEcNyuE7hDq8Ei5vA on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-embedding-ada-002 in organization org-pVOnO8dpEcNyuE7hDq8Ei5vA on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..
Retrying langcha

RateLimitError: Rate limit reached for text-embedding-ada-002 in organization org-pVOnO8dpEcNyuE7hDq8Ei5vA on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.

In [146]:
tfidf_retriever = TFIDFRetriever.from_texts(splits)

In [147]:
question="what did they say about matlab?"

In [148]:
docs = tfidf_retriever.get_relevant_documents(question)
docs

[Document(page_content="Saxena and Min Sun here did, wh ich is given an image like this, right? This is actually a \npicture taken of the Stanford campus. You can apply that sort of cl ustering algorithm and \ngroup the picture into regions. Let me actually blow that up so that you can see it more \nclearly. Okay. So in the middle, you see the lines sort of groupi ng the image together, \ngrouping the image into [inaudible] regions.  \nAnd what Ashutosh and Min did was they then  applied the learning algorithm to say can \nwe take this clustering and us e it to build a 3D model of the world? And so using the \nclustering, they then had a lear ning algorithm try to learn what the 3D structure of the \nworld looks like so that they could come up with a 3D model that you can sort of fly \nthrough, okay? Although many people used to th ink it's not possible to take a single \nimage and build a 3D model, but using a lear ning algorithm and that sort of clustering \nalgorithm is the first st